# Electronic Benefits Verification

When a patient is prescribed their medication, often times they will use some sort of commercial insurance in order to pay for the product. The patient may also have a government funcded insurance plan. Patients on a government funded plan will often require different channels of work that are outside the scope of a Customer Support Program.

The goal of an eBV is to provide an efficient method for allowing providers to verify a patient's coverage while in the office and reducing delays in time to therapy.

What we are testing here is an integration with a company called [sonexus](https://www.cardinalhealth.com/en/services/manufacturer/biopharmaceutical/patient-access-and-adherence.html).

#

In [ ]:
"""
running the initial query to grab test claims

@author Preston Mackert
"""
# -------------------------------------------------------------------------------------------------------------------- #
# imports
# -------------------------------------------------------------------------------------------------------------------- #

from pcp_rest_client import sfdc_interface
from pcp_rest_client import sfdc_query_utility as queries
from pcp_rest_client import pcp_program_utility as program_util
import pandas as pd


# -------------------------------------------------------------------------------------------------------------------- #
# support methods
# -------------------------------------------------------------------------------------------------------------------- #

def structure_ebv_file(filename):
    """ reads in the eBv test csv file sent from sonexus """
    ebv_file = pd.read_csv(filename)
    programs = {}
    for index, row in ebv_file.iterrows():
        """ mapping data back into PCP program level data """
        # writing out each column to get a better look... this is the field mapping from Sonexus
        # program_id = row['HUB ID']
        program_id = row['ExternalRefid']
        first_name = row['PatientFirstName']
        last_name = row['PatientLastName']
        gender = row['PatientGender']
        dob = row['PatientDoB']
        address_1 = row['PatientAddressLine1']
        address_2 = row['PatientAddressLine2']
        city = row['PatientAddressCity']
        state = row['PatientAddressState']
        zip = row['PatientAddressPostalCode']
        doc_npi = row['DrNameWithNPI']
        payer = row['Payername']
        plan_number = row['PlanNumber']
        card_holder_number = row['CardholderNumber']
        card_holder_first_name = row['CardholderFirstName']
        card_holder_last_name = row['CardholderLastName']
        card_holder_relationship = row['CardholderRelationship']
        card_holder_relationship_code = row['CardholderRelationshipCode']
        health_plan_date = row['HealthPlanDate']
        group_number_group_name = row['GroupNumberGroupName']
        insurance_plan_name = row['InsurancePlanName']
        formulary_id = row['FormularyID']
        bin = row['BIN']
        processor_control_number = row['ProcessorControlNumber']
        # medicare_indicator = row['MedicareIndicator']
        eligible_retail_pharmacy = row['EligibleRetailPharmacy']
        eligible_mailorder_pharmacy = row['EligibleMailOrderPharmacy']
        eligible_specialty_pharmacy = row['EligibleSpecialtyPharmacy']
        eligible_90_day_retail_pharmacy = row['Eligible90DayRetailPharmacy']
        eligible_LTC = row['EligibleLTC']
        drug_status_code = row['DrugStatusCode']
        quantity_priced = row['QuantityPriced']
        days_supply = row['DaysSupply']
        prior_auth_required = row['PriorAuthRequired']
        reason_code = row['ReasonCode']
        reason_code_freetext = row['ReasonCodeFreeText']
        pharmacy_type = row['PharmacyType']
        pharmacy_npi = row['PharmacyNpi']
        ncpdp_id = row['NCPDPID']
        reference_text = row['ReferenceText']
        estimated_patient_pay_amount = row['EstimatedPatientPayAmount']
        plan_pay_amount = row['PlanPayAmount']
        oop_applied_amount = row['OopAppliedAmount']
        oop_remaining_amount = row['OopRemainingAmount']
        deductible_applied_amount = row['DeductibleAppliedAmount']
        deductible_remaining_amount = row['DeductibleRemainingAmount']
        # creating entry in map
        programs[program_id] = {
            # 'reference_id': ref_id,
            'first_name': first_name,
            'last_name': last_name,
            'gender': gender,
            'date_of_birth': dob,
            'address_line_1': address_1,
            'address_line_2': address_2,
            'city': city,
            'state': state,
            'zip_code': zip,
            'doc_npi': doc_npi,
            'payer': payer,
            'plan_number': plan_number,
            'cardholder_number': card_holder_number,
            'cardholder_first_name': card_holder_first_name,
            'cardholder_last_name': card_holder_last_name,
            'cardholder_relationship': card_holder_relationship,
            'cardholder_relationship_code': card_holder_relationship_code,
            'health_plan_date': health_plan_date,
            'group_number_group_name': group_number_group_name,
            'insurance_plan_name': insurance_plan_name,
            'formulary_id': formulary_id,
            'bin': bin,
            'processor_control_number': processor_control_number,
            # 'medicare_indicator': medicare_indicator,
            'eligible_retail_pharmacy': eligible_retail_pharmacy,
            'eligible_mailorder_pharmacy': eligible_mailorder_pharmacy,
            'eligible_specialty_pharmacy': eligible_specialty_pharmacy,
            'eligible_90_day_retail_pharmacy': eligible_90_day_retail_pharmacy,
            'eligible_LTC': eligible_LTC,
            'drug_status_code': drug_status_code,
            'quantity_priced': quantity_priced,
            'days_supply': days_supply,
            'prior_auth_required': prior_auth_required,
            'reason_code': reason_code,
            'reason_code_freetext': reason_code_freetext,
            'pharmacy_type': pharmacy_type,
            'pharmacy_npi': pharmacy_npi,
            'ncpdp_id': ncpdp_id,
            'reference_text': reference_text,
            'estimated_patient_pay_amount': estimated_patient_pay_amount,
            'plan_pay_amount': plan_pay_amount,
            'oop_applied_amount': oop_applied_amount,
            'oop_remaining_amount': oop_remaining_amount,
            'deductible_applied_amount': deductible_applied_amount,
            'deductible_remaining_amount': deductible_remaining_amount
        }
    # returning the new data structure with programs as key
    return programs


def grab_sf_data(sf, ebv_programs):
    """ takes in a list of programs from the eBv file and queries the program ids in sf to access coverage data """
    programs = list(ebv_programs.keys())
    # Taltz decided to give us a nan value for a program id :)
    programs = [program for program in programs if str(program) != 'nan']
    soql = "SELECT Name, " \
           "(" \
           "SELECT RecordTypeId, Id, Name, DTPC_Payer_Name__r.Name, DTPC_Plan_Name__c, DTPC_PBM_Name__r.Name, " \
           "DTPC_Group_Number__c, DTPC_ID_Policy_Number__c, DTPC_BIN__c, DTPC_PCN__c, DTPC_Record_Status__c " \
           "FROM Coverage1__r" \
           "), " \
           "(" \
           "SELECT Id, Name, Coverage_Status__c " \
           "FROM Coverage__r" \
           ") " \
           "FROM " \
           "DTPC_Program__c " \
           "WHERE Name IN "
    # the list is partitioned through the "query large list" functino :)
    records = queries.query_list(sf, soql, programs)
    clean_map = {}
    for program in records:
        new_coverage_records = program_util.parse_coverages(program)
        copay_coverages = program_util.filter_active_coverages(new_coverage_records[0])
        insurance_coverages = program_util.filter_active_coverages(new_coverage_records[1])
        old_coverage_records = program_util.parse_old_coverages(program)
        old_coverage_records = program_util.filter_active_old_coverages(old_coverage_records)
        clean_map[program['Name']] = [{'copay': copay_coverages, 'insurance': insurance_coverages},
                                      old_coverage_records]
    return clean_map


def print_general_stats(ebv_programs, salesforce_data, brand):
    # abstracted data assets
    status_set = {}
    pa_set = {}
    combinations = {}
    ebv_payer_names = {}
    group_numbers = []

    for program in salesforce_data:
        try:
            # defining the coverage record lists
            insurance_coverages = salesforce_data[program][0]['insurance']
            old_coverages = salesforce_data[program][1]

            # setting ebv variables for matching
            status_code = ebv_programs[program]['drug_status_code']
            pa_required = ebv_programs[program]['prior_auth_required']
            payer_name = ebv_programs[program]['payer']

            if payer_name in list(ebv_payer_names.keys()):
                ebv_payer_names[payer_name] += 1
            else:
                ebv_payer_names[payer_name] = 1

            # coverage variables for matching
            for coverage in insurance_coverages:
                group_number = coverage['DTPC_Group_Number__c']
                group_numbers.append(group_number)

            # adding data for abstracted statistics
            if status_code in list(status_set.keys()):
                status_set[status_code] += 1
            else:
                status_set[status_code] = 1

            if pa_required in list(pa_set.keys()):
                pa_set[pa_required] += 1
            else:
                pa_set[pa_required] = 1

            if status_code in list(combinations.keys()):
                if pa_required not in combinations[status_code]:
                    combinations[status_code].append(pa_required)
            else:
                combinations[status_code] = [pa_required]
        except:
            continue

    # analyzing status combinations...
    print("\ngeneral %s statistics" % brand)
    print("---------------------------")
    print("assessing the different status combinations that can be returned...")
    print("drug status codes: ", status_set)
    print("pa required codes: ", pa_set)
    print("combinations: ", combinations)
    print("identified payers: ", ebv_payer_names)
    print("identified group numbers: ", set(group_numbers))


# -------------------------------------------------------------------------------------------------------------------- #
# main
# -------------------------------------------------------------------------------------------------------------------- #

def main():
    print("connecting to salesforce...")
    sf = sfdc_interface.connect_to_prod(True)

    print("\nreading in forteo eBv test file...")
    forteo_ebv_programs = structure_ebv_file('test_files/FORTEO_eBV_Batch_Datapull_01062020.csv')
    print("number of programs evaluated: ", len(forteo_ebv_programs))
    salesforce_data = grab_sf_data(sf, forteo_ebv_programs)
    print_general_stats(forteo_ebv_programs, salesforce_data, "forteo")

    print("\n\nreading in olumiant eBv test file...")
    olumiant_ebv_programs = structure_ebv_file(
        'test_files/Olumiant_eBV_Batch_Datapull_01062020 Excluding Express Scripts.csv')
    print("number of programs evaluated: ", len(olumiant_ebv_programs))
    salesforce_data = grab_sf_data(sf, olumiant_ebv_programs)
    print_general_stats(olumiant_ebv_programs, salesforce_data, "olumiant")

    print("\n\nreading in Taltz eBv test file...")
    taltz_ebv_programs = structure_ebv_file('test_files/Taltz_eBV_Batch_Datapull_01082020 - First File.csv')
    print("number of programs evaluated: ", len(taltz_ebv_programs))
    salesforce_data = grab_sf_data(sf, taltz_ebv_programs)
    print_general_stats(taltz_ebv_programs, salesforce_data, "taltz")


# running the script
main()
